In [1]:
import pandas as pd
import numpy as np

pr_gen_fuel = pd.read_pickle('../data/raw_eia923__puerto_rico_generation_fuel')
pr_plant_frame = pd.read_pickle('../data/raw_eia923__puerto_rico_plant_frame')

In [ ]:
# Handle EIA null values
pr_gen_fuel = pr_gen_fuel.replace(to_replace = ".", value = pd.NA)
# Convert data types (mmbtu/units to numeric)
pr_gen_fuel = pr_gen_fuel.convert_dtypes()

In [ ]:
# Pivot fuel_consumed_for_electricity columns
## Only keep the index and relevant fuel_consumed columns
column_subset_list = ['plant_id_eia', 'plant_name_eia', 'report_year', 'prime_mover_code', 'energy_source_code'] + [col for col in pr_gen_fuel.columns if "fuel_consumed_for_electricity_mmbtu" in col]
pr_gen_fuel_elec = pr_gen_fuel.loc[:, column_subset_list]

## Melt the fuel_consumed columns
pr_gen_fuel_elec_melt = pr_gen_fuel_elec.melt(id_vars = ['report_year', 'plant_id_eia', 'plant_name_eia', 'prime_mover_code', 'energy_source_code'])
## Split the month from the variable
pr_gen_fuel_elec_melt[['fuel_variable', 'month']] = pr_gen_fuel_elec_melt['variable'].str.rsplit("_", n=1, expand=True)
## Create date from month and year
pr_gen_fuel_elec_melt['date'] = pd.to_datetime(pr_gen_fuel_elec_melt['month'] + pr_gen_fuel_elec_melt['report_year'].astype(str), format='%B%Y')
## Drop some stuff
pr_gen_fuel_elec_clean = pr_gen_fuel_elec_melt.drop(columns = ['report_year', 'month', 'variable', 'fuel_variable'])

In [ ]:
## Drop a bad plant
pr_gen_fuel_elec_final = pr_gen_fuel_elec_clean.loc[~((pr_gen_fuel_elec_clean.plant_id_eia == 62410) & (pr_gen_fuel_elec_clean.date.dt.year == 2020) & (pr_gen_fuel_elec_clean.value.isnull()))]

## Rename columns
pr_gen_fuel_elec_final = pr_gen_fuel_elec_final.rename(columns={'value':'fuel_consumed_for_electricity_mmbtu'})

In [189]:
pr_gen_fuel_elec_final

,plant_id_eia,plant_name_eia,prime_mover_code,energy_source_code,fuel_consumed_for_electricity_mmbtu,date
0,61014,Pattern Santa Isabel LLC,WT,WND,101260.0,2017-04-01
1,61034,EcoElectrica,CA,NG,0.0,2017-04-01
2,61034,EcoElectrica,CT,NG,1976130.0,2017-04-01
3,61036,AES ILUMINA,PV,SUN,31886.0,2017-04-01
4,61082,AES Puerto Rico,ST,BIT,3258736.0,2017-04-01
...,...,...,...,...,...,...
5395,61149,Palo Seco Plant,GT,DFO,<NA>,2025-09-01
5396,61149,Palo Seco Plant,ST,RFO,<NA>,2025-09-01
5397,61150,Cambalache Plant,GT,DFO,<NA>,2025-09-01
5398,61151,Mayaguez Plant,GT,DFO,<NA>,2025-09-01
